In [6]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import sys 
sys.path.append("..")
from utils import *

In [7]:
INPUT = "../input"
TRAIN = os.path.join(INPUT, 'train_data.csv')
TEST = os.path.join(INPUT, 'test_data.csv')
FEATURE = os.path.join(INPUT, "feature")
ALL = os.path.join(INPUT, 'all.csv')
SUBMIT = "../submit"
target = ["y"]

In [8]:
df_train = read_csv(TRAIN)
df_test = read_csv(TEST)
# 初回のみ
#all_df = train_test_merge(train_df, test_df)
#二回目以降
df = read_new_csv()
df_train, df_test = df_split(df)
#date_df = read_csv(date)

../feature/08161926.csv was read


In [4]:
#df_all = read_csv(ALL)
#train_df_,test_df_ = df_split(df_all)
df_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,...,duration_log,duration_sqrt,balance_log,balance_sqrt,balance_log_cut,age_bin,campaign_log,all_campaign,campaign_bin,previous_bin
0,58,1,1,1,0,10163,1,0,1,5,...,5.568345,16.186414,9.226509,100.811706,0.226509,3,0.0,1,1,0
1,44,2,2,2,0,8049,1,0,1,5,...,5.023881,12.328828,8.993303,89.716219,-0.006697,3,0.0,1,1,0
2,33,3,1,2,0,8022,1,1,1,5,...,4.343805,8.774964,8.989943,89.565618,-0.010057,3,0.0,1,1,0
3,33,4,2,3,0,8021,0,0,1,5,...,5.293305,14.106736,8.989818,89.560036,-0.010182,3,0.0,1,1,0
4,35,1,1,1,0,8251,1,0,1,5,...,4.941642,11.832160,9.018090,90.835015,0.018090,3,0.0,1,1,0


## データの分割

In [19]:
from  sklearn.model_selection  import train_test_split
#using = ["age", "job", "marital", "education", "default", "balance_log_cut", "housing", "loan","contact",  "duration_log", "campaign_bin", "pdays", "previous_bin", "poutcome", "month_day", "is_job_kesson","is_education_kesson","is_poutcome_kesson","is_contact_kesson" ]
using = ['duration_log','month_day','balance_log_cut','age','poutcome','job','pdays','campaign_bin','housing','education','marital']
y = df_train[target].values
X = df_train[using].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)
y

array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]])

# モデル

In [20]:
SEED = 10
# 必要なもののインポート
## 予測モデル
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb


## base line with RFC　（0.９３２）

In [21]:
rf = RandomForestClassifier()
rf.fit(X,y)

<ipython-input-21-e598faf66ab5>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X,y)


RandomForestClassifier()

In [22]:
prd = rf.predict_proba(df_test[using])
prd_df = pd.DataFrame(prd)
df_prd = prd_df.iloc[:, [1]]

In [23]:
to_submit_csv(df_prd)

../submit/08161932_submit.csv
save completed


In [88]:
prd_df.iloc[:, [1]]
#prd_df.iloc(:,[1])

,1
0,0.02
1,0.00
2,0.00
3,0.00
4,0.01
...,...
4995,0.55
4996,0.65
4997,0.15
4998,0.69


In [18]:
from pprint import pprint
fti = rf.feature_importances_ 
feature_list = list(zip(using, fti))
feature_list.sort(key=lambda x: x[1], reverse = True)
max_len_feature = max(map( lambda s: len(s), using))
print('Feature Importances:')
#pprint(feature_list)
for tp in feature_list:
    print("{:{}}:{:.3}".format(tp[0],max_len_feature+1, tp[1]))
print("top10")
for s in feature_list[0:11]:
    print(f"'{s[0]}',",end = '')

Feature Importances:
duration_log        :0.296
month_day           :0.153
balance_log_cut     :0.114
age                 :0.107
poutcome            :0.0621
job                 :0.0511
pdays               :0.0437
campaign_bin        :0.0378
housing             :0.0249
education           :0.022
marital             :0.0206
previous_bin        :0.0196
is_contact_kesson   :0.0133
loan                :0.00935
is_poutcome_kesson  :0.00842
contact             :0.00804
is_education_kesson :0.00588
default             :0.0019
is_job_kesson       :0.00146
top10
'duration_log','month_day','balance_log_cut','age','poutcome','job','pdays','campaign_bin','housing','education','marital',

## base line with LGBM（0.938）

In [109]:
params = {
    # 二値分類問題
    'objective': 'binary',
    # AUC の最大化を目指す
    'metric': 'auc',
    # Fatal の場合出力
    'verbosity': -1,
    
}
lgb_train = lgb.Dataset(X, y)

In [113]:
model = lgb.train(params, lgb_train,
                  verbose_eval=50,  # 50イテレーション毎に学習結果出力
                  num_boost_round=100,  # 最大イテレーション回数指定
                 )

In [114]:
y_pred_lgb = model.predict(df_test[using])

In [115]:
prd_df = pd.DataFrame(y_pred_lgb)
#df_prd = prd_df.iloc[:, [1]]
to_submit_csv(prd_df)

../submit/8161049_submit.csv
save completed


In [116]:
df_prd = prd_df.iloc[:, [0]]
boo = np.squeeze(np.array(df_prd))

hoo = np.add(boo, y_pred_lgb)/2

In [117]:
to_submit_csv(pd.DataFrame(hoo))

../submit/8161049_submit.csv
save completed
